<h1> Классификация спама в e-mail сообщениях

<h2> Постановка задачи:</h2>
<p>Дан датасет - сообщения и пометка, является ли это сообщение спамом (957 сообщений на тренировочной выборке и 125 на тестовой). Цель задачи - научиться автоматически помечать сообщения со спамом.</p>

<h3> Читаем тренировочные данные

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np          
import seaborn as sns         
import sklearn
import nltk
from nltk.corpus import stopwords
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')


train_dataset = []
test_dataset = []
dataset = []

train_dataset = pd.read_csv("archive/SMS_train.csv", sep=',', encoding="Windows-1252")
test_dataset = pd.read_csv("archive/SMS_test.csv", sep=',', encoding="Windows-1252")

dataset = pd.concat((train_dataset, test_dataset))
#dataset = train_dataset

<h3> Взглянем на информацию о датасете

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset.shape

In [ ]:
dataset.head(10)

In [ ]:
dataset.tail(10)

In [ ]:
dataset.info()

<h3> Получаем набор всех слов встречающихся в выборке (Составляем "Мешок слов")

In [ ]:
def gram_tokens(tokens, gram = 2):
    if gram > 1:
        t = []
        for i in range(len(tokens) - gram + 1): 
            t += [" ".join(tokens[i:i + gram])]
        return t

def process_tokens(tokens, stem = True, stop_words = True):
    if stop_words:
        sw = stopwords.words("english")
        tokens = [token for token in tokens if token not in sw]
    if stem:
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    return tokens

In [ ]:
tokens = []


for xl in dataset["Message_body"]:
    token = nltk.word_tokenize(str(xl).lower())
    token = [t for t in token if len(t) > 2]
    for t in token:
        tokens.append(t)
tokens = set(tokens)
tokens = list(tokens)

In [ ]:
#tokens = gram_tokens(tokens, 2)
tokens = process_tokens(tokens, False, False)

In [ ]:
#количество неповторяющихся слов
tokens.__len__()

<h3> Обрабатываем датасет. Теперь для каждой записи указано сколько раз встречается то или иное слово в сообщении

In [51]:
def bag_of_words(tokens, dataset):
    df = pd.DataFrame(dataset)
    for index, row in dataset.iterrows():
        if index == 113:
            print(row["Message_body"].lower())
        msg_tokens = nltk.word_tokenize(str(row["Message_body"]).lower())
        msg_tokens = [t for t in msg_tokens if len(t) > 2]
        #msg_tokens = gram_tokens(msg_tokens, 2)
        msg_tokens = process_tokens(msg_tokens, False, False)
        count_letter = Counter(msg_tokens)
        for token in tokens:
            if token in df.columns:
                df[df.index == index][token] += count_letter[token]
            else: 
                df.loc[index, token] = count_letter[token]
    return df.copy()

df = bag_of_words(tokens, dataset)


df.drop("Message_body", axis=1, inplace=True)

KeyboardInterrupt: 

In [ ]:
df.to_csv("new_data.csv")

In [ ]:
df.drop("S. No.", axis=1, inplace=True)

<h3> Вот так теперь выглядит датасет

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.iloc[:,0] = df.iloc[:,0].astype('category')

In [ ]:
df.info()

<h3> Сводная таблица по количественным признакам встречающихся слов

In [ ]:
df.describe()

<h3> Сводная таблица по категориям сообщений

In [ ]:
df.describe(include = ['category'])

<h3> Строим корреляционную матрицу

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, square=True, cmap='coolwarm')

Визуализировав корреляционную матрицу мы видим связь между некоторыми словами. Подобное происходит оттого, что в человеческой речи слова связаны друг с другом по смыслу.
На данный момент мы не можем воспользоваться этой информацией, поэтому попробуем найти корреляцию слов от типа сообщения

<h3> Визуализируем какие слова чаще всего используются в сообщениях со спамом

Для поиска таких слов воспользуемся библиотекой WordCloud

In [ ]:
from wordcloud import WordCloud

text = " ".join(list(dataset[dataset["Label"] == "Spam"]["Message_body"]))

# Generate a word cloud image
wordcloud = WordCloud(width = 512,height = 512).generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
ham_words = ' '.join(list(dataset[dataset["Label"] == "Non-Spam"]["Message_body"]))
ham_wc = WordCloud(width = 512,height = 512).generate(ham_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(ham_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

<h3> Нормализуем данные

In [ ]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'category']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'category']

In [ ]:
for col_num in numerical_columns:
    if  not df[col_num].std(axis = 0):
        print(df[col_num])
        df.drop(col_num, axis=1, inplace=True)
        numerical_columns.remove(col_num)

In [ ]:
data_numerical = df[numerical_columns]
data_numerical.describe()

In [ ]:
data_numerical = (data_numerical - data_numerical.mean(axis = 0))/data_numerical.std(axis = 0)
data_numerical.describe()

In [ ]:
data = pd.concat((data_numerical, df['Label']), axis = 1)

In [ ]:
data

In [ ]:
X = data.drop('Label', axis = 1)
y = data['Label']

<h3> Повторим всю обработку для тестовой выборки

test_dataset = pd.read_csv("archive/SMS_test.csv", sep=',', encoding="Windows-1252")

df2 = tokenization(tokens, test_dataset)

df2.drop("Message_body", axis=1, inplace=True)
df2.drop("S. No.", axis=1, inplace=True)

df2.iloc[:,0] = df2.iloc[:,0].astype('category')

df2.head(10)

numerical_columns_test = [c for c in df2.columns if df2[c].dtype.name != 'category']

print(numerical_columns_test)

data_numerical_test = df2[numerical_columns_test]

print(data_numerical_test)

data_numerical_test = (data_numerical_test - data_numerical_test.mean(axis = 0))/data_numerical_test.std(axis = 0)

print(data_numerical_test)

data2 = pd.concat((data_numerical_test, df2['Label']), axis = 1)

X_test = data2.drop('Label', axis = 1)
y_test = data2['Label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 50)

In [ ]:
X_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import ensemble

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_train, y_train)
print(knn)

err_train = np.mean(y_train != knn.predict(X_train))
err_test = np.mean(y_test != knn.predict(X_test))

print('Ошибка на обучающей выборке: ', err_train)
print('Ошибка на тестовой выборке: ', err_test)